In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report,roc_auc_score

In [2]:
concern_cols = ['origins', 'lockdowns', 'masking', 'healthcare', 'education', 'therapeutics', 'vaccines']

dfs = {}

annot_dir = '../covid_data/concern_annotations/'

dfs['Nina'] = pd.read_csv(annot_dir+'nina.csv',lineterminator='\r')
dfs['Nina'] = dfs['Nina'].fillna(value=0)
dfs['Nina'] = dfs['Nina'][['index','tweetid','text']+concern_cols]

dfs['Fiona'] = pd.read_csv(annot_dir+'guo.csv',lineterminator='\r')
dfs['Fiona'] = dfs['Fiona'].fillna(value=0)
dfs['Fiona'] = dfs['Fiona'][['index','tweetid','text']+concern_cols]

dfs['Ashwin'] = pd.read_csv(annot_dir+'Ashwin.csv',lineterminator='\n')
dfs['Ashwin'] = dfs['Ashwin'][['index','tweetid','text']+concern_cols]

dfs['Kristina'] = pd.read_csv(annot_dir+'KL.csv',lineterminator='\n')
dfs['Kristina'] = dfs['Kristina'][['index','tweetid','text']+concern_cols]
dfs['Kristina'][concern_cols] = dfs['Kristina'][concern_cols].where(dfs['Kristina'][concern_cols]!=0.5,1)

dfs['Nina_RA'] = pd.read_excel(annot_dir+'nina_RA.xlsx')
dfs['Nina_RA'][concern_cols] = dfs['Nina_RA'][concern_cols].where(dfs['Nina_RA'][concern_cols]==1,0)

for k in dfs.keys():
    dfs[k] = dfs[k].sort_values('index')

# Cohen's Kappa

In [5]:
cohens = {}
for c in concern_cols:
    cohen_kappa = pd.DataFrame(index=list(dfs.keys()),columns= list(dfs.keys()))
    for i in list(dfs.keys()):
        for j in list(dfs.keys()):
            cohen_kappa.loc[i,j] = cohen_kappa_score(dfs[i][c].astype(int).tolist(),dfs[j][c].astype(int).tolist())
    cohens[c] = cohen_kappa

In [15]:
import numpy as np

for k,v in cohens.items():
    print(k," - Cohen's Kappa")
    print(v)
    print()
    tmp = [i for i in np.triu(v.values,k=1).reshape(-1) if i!=0]
    print(np.mean(tmp),np.std(tmp))
    print()

origins  - Cohen's Kappa
              Nina     Fiona    Ashwin  Kristina   Nina_RA
Nina           1.0  0.736639  0.830799  0.827763  0.676379
Fiona     0.736639       1.0  0.733943  0.736639  0.621523
Ashwin    0.830799  0.733943       1.0  0.752706   0.72704
Kristina  0.827763  0.736639  0.752706       1.0  0.703347
Nina_RA   0.676379  0.621523   0.72704  0.703347       1.0

0.7346776319841053 0.05956509520015955

lockdowns  - Cohen's Kappa
              Nina     Fiona    Ashwin  Kristina   Nina_RA
Nina           1.0  0.772031  0.822271  0.804718   0.77572
Fiona     0.772031       1.0  0.773654  0.755584  0.703707
Ashwin    0.822271  0.773654       1.0  0.840538  0.844431
Kristina  0.804718  0.755584  0.840538       1.0  0.833971
Nina_RA    0.77572  0.703707  0.844431  0.833971       1.0

0.7926624588080513 0.04249980011795256

masking  - Cohen's Kappa
              Nina     Fiona    Ashwin  Kristina   Nina_RA
Nina           1.0  0.927634  0.948599  0.875196  0.960739
Fiona     0.927

# Fleiss' Kappa (multi-annotator)

In [104]:
def fleiss_kappa(M):
    """Computes Fleiss' kappa for group of annotators.
    :param M: a matrix of shape (:attr:'N', :attr:'k') with 'N' = number of subjects and 'k' = the number of categories.
        'M[i, j]' represent the number of raters who assigned the 'i'th subject to the 'j'th category.
    :type: numpy matrix
    :rtype: float
    :return: Fleiss' kappa score
    """
    N, k = M.shape  # N is # of items, k is # of categories
    n_annotators = float(np.sum(M[0, :]))  # # of annotators
    tot_annotations = N * n_annotators  # the total # of annotations
    category_sum = np.sum(M, axis=0)  # the sum of each category over all items

    # chance agreement
    p = category_sum / tot_annotations  # the distribution of each category over all annotations
    PbarE = np.sum(p * p)  # average chance agreement over all categories

    # observed agreement
    P = (np.sum(M * M, axis=1) - n_annotators) / (n_annotators * (n_annotators - 1))
    Pbar = np.sum(P) / N  # add all observed agreement chances per item and divide by amount of items

    return round((Pbar - PbarE) / (1 - PbarE), 4)

In [123]:
df_agg = dfs['Nina']
df_agg[concern_cols] = dfs['Nina'][concern_cols] + dfs['Fiona'][concern_cols] + dfs['Ashwin'][concern_cols] + dfs['Kristina'][concern_cols] + dfs['Nina_RA'][concern_cols]

In [124]:
df_agg

,index,tweetid,text,origins,lockdowns,masking,healthcare,education,therapeutics,vaccines
0,0,1.320000e+18,"A very important read. ""To stop the next pande...",4.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1.300000e+18,I do my best to understand conspiracy theories...,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1.330000e+18,I hope @JoeBiden &amp; @BorisJohnson use their...,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,3,1.230000e+18,Oregon #coronavirus case of unknown origin con...,2.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,1.360000e+18,"I'm not against people with conspiracy, I'm fi...",3.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
779,779,1.400000e+18,🚨@MattHancock as you host #G7 Health Ministers...,0.0,0.0,0.0,0.0,0.0,0.0,4.0
780,780,1.340000e+18,Pediatricians Want Kids to Be Part of COVID Va...,0.0,0.0,1.0,1.0,0.0,0.0,4.0
781,781,1.360000e+18,Iran cleric: People who are vaccinated for COV...,0.0,0.0,1.0,0.0,0.0,0.0,4.0
782,782,1.430000e+18,Fauci: Coronavirus 'Will Disappear' When an 'M...,0.0,0.0,0.0,2.0,0.0,0.0,4.0


In [125]:
fleiss = {}
for c in concern_cols:
    pos = df_agg[c].tolist()
    neg = [5 - i for i in pos]
    fleiss[c] = fleiss_kappa(np.array([pos,neg]).T)
print("Fleiss' Kappa for Multi-annotators:")
fleiss

Fleiss' Kappa for Multi-annotators:


{'origins': 0.4468,
 'lockdowns': 0.4675,
 'masking': 0.5478,
 'healthcare': 0.2692,
 'education': 0.4936,
 'therapeutics': 0.3987,
 'vaccines': 0.5318}

# F1 score for Model Predictions

In [149]:
df_majority = df_agg.copy()
df_majority[concern_cols] = df_majority[concern_cols].where(df_majority[concern_cols]>=3,0)
df_majority[concern_cols] = df_majority[concern_cols].where(df_majority[concern_cols]<3,1)
# df_majority = df_majority.sort_values(['tweetid'])
df_majority = df_majority.drop_duplicates(subset=['text'])
df_majority = df_majority.sort_values(['index'])
df_majority.set_index('index',inplace=True)
df_majority

,tweetid,text,origins,lockdowns,masking,healthcare,education,therapeutics,vaccines
index,,,,,,,,,
0,1.320000e+18,"A very important read. ""To stop the next pande...",1,0,0,0,0,0,0
1,1.300000e+18,I do my best to understand conspiracy theories...,0,0,0,0,0,0,0
2,1.330000e+18,I hope @JoeBiden &amp; @BorisJohnson use their...,0,0,0,0,0,0,1
3,1.230000e+18,Oregon #coronavirus case of unknown origin con...,0,0,0,0,0,0,0
4,1.360000e+18,"I'm not against people with conspiracy, I'm fi...",1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
778,1.390000e+18,@DeraSachaSauda volunteers are supporting the ...,0,0,0,1,0,0,0
779,1.400000e+18,🚨@MattHancock as you host #G7 Health Ministers...,0,0,0,0,0,0,1
780,1.340000e+18,Pediatricians Want Kids to Be Part of COVID Va...,0,0,0,0,0,0,1


In [152]:
gt_concern_cols = ['origins_gt', 'lockdowns_gt', 'masking_gt', 'healthcare_gt',
       'education_gt', 'therapeutics_gt', 'vaccines_gt']

df_preds = pd.read_csv('../covid_784_sample_for_annotation.csv',lineterminator='\n')
df_preds = df_preds[['index','tweetid','text']+concern_cols]
df_preds.columns = ['index','tweetid', 'text', 'origins_gt', 'lockdowns_gt', 'masking_gt', 'healthcare_gt',
       'education_gt', 'therapeutics_gt', 'vaccines_gt']
df_preds = df_preds.loc[df_majority.index,]
df_preds

,index,tweetid,text,origins_gt,lockdowns_gt,masking_gt,healthcare_gt,education_gt,therapeutics_gt,vaccines_gt
index,,,,,,,,,,
0,0,1.320000e+18,"A very important read. ""To stop the next pande...",1,0,0,0,0,0,0
1,1,1.300000e+18,I do my best to understand conspiracy theories...,1,0,0,0,0,0,0
2,2,1.330000e+18,I hope @JoeBiden &amp; @BorisJohnson use their...,1,0,0,0,0,0,1
3,3,1.230000e+18,Oregon #coronavirus case of unknown origin con...,1,0,0,0,0,0,0
4,4,1.360000e+18,"I'm not against people with conspiracy, I'm fi...",1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
778,778,1.390000e+18,@DeraSachaSauda volunteers are supporting the ...,0,0,0,0,0,0,1
779,779,1.400000e+18,🚨@MattHancock as you host #G7 Health Ministers...,0,0,0,0,0,0,1
780,780,1.340000e+18,Pediatricians Want Kids to Be Part of COVID Va...,0,0,0,0,0,0,1


In [164]:
print('F1-scores:')
print(classification_report(df_majority[concern_cols].astype(int).values,df_preds[gt_concern_cols].astype(int).values,target_names=concern_cols))

auc_scores = roc_auc_score(df_majority[concern_cols].astype(int).values,df_preds[gt_concern_cols].astype(int).values,average=None)
print('\n\nROC AUC scores:\n')
for i in range(len(concern_cols)):
    print(concern_cols[i],": ",round(auc_scores[i],2))

F1-scores:
              precision    recall  f1-score   support

     origins       0.35      0.92      0.51        25
   lockdowns       0.86      0.94      0.90       108
     masking       0.71      0.99      0.83       101
  healthcare       0.32      0.91      0.47        46
   education       0.59      1.00      0.74        54
therapeutics       0.20      0.90      0.32        20
    vaccines       0.90      0.94      0.92        84

   micro avg       0.57      0.95      0.72       438
   macro avg       0.56      0.94      0.67       438
weighted avg       0.68      0.95      0.77       438
 samples avg       0.59      0.67      0.62       438



ROC AUC scores:

origins :  0.92
lockdowns :  0.95
masking :  0.95
healthcare :  0.87
education :  0.96
therapeutics :  0.88
vaccines :  0.96


/Users/fionaguo/.pyenv/versions/3.9.10/envs/disentangle_emotions/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
roc_auc_score([[0,1,0],[1,1,1],[0,0,1]],[[1,1,0],[1,1,1],[0,1,1]],average=None)

array([0.75, 0.5 , 1.  ])